In [8]:
import pandas as pd
import time
import numpy as np
from nba_api.stats.endpoints import playercareerstats
import nba_api.stats.endpoints as nbaapi
from nba_api.stats.endpoints import leaguegamefinder
import psycopg2
import re
import allintwo

conn = allintwo.connect_to_rds('thebigone', 'ajwin', 'CharlesBark!23', 'nba-rds-instance.c9wwc0ukkiu5.us-east-1.rds.amazonaws.com')


Connected to RDS PostgreSQL database


In [11]:
    
query = "SELECT * FROM pg_catalog.pg_tables where tableowner = 'ajwin';"  # Example query: fetch 10 rows from the 'games' table

if conn:
    result_df = allintwo.query_database_to_dataframe(conn, query)

result_df

c:\Users\ajwin\Projects\NBA\thebigone\allintwo.py:103: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dataframe = pd.read_sql(query, conn)


,schemaname,tablename,tableowner,tablespace,hasindexes,hasrules,hastriggers,rowsecurity
0,public,masterplayers,ajwin,None,False,False,False,False
1,public,mastergames,ajwin,None,False,False,False,False
2,public,boxscoreadvancedv3_0,ajwin,None,False,False,False,False


In [9]:
    
query = "SELECT * FROM mastergames;"  # Example query: fetch 10 rows from the 'games' table

result_df = allintwo.query_database_to_dataframe(conn, query)

result_df


,seasonid,teamid,teamabbreviation,teamname,gameid,gamedate,matchup,wl,min,pts,...,ftpct,oreb,dreb,reb,ast,stl,blk,tov,pf,plusminus
0,22024,1610612749,MIL,Milwaukee Bucks,0022400145,2024-11-02,MIL vs. CLE,None,121,71,...,0.455,5.0,21.0,26,16,2.0,2,7,13,1.0
1,22024,1610612759,SAS,San Antonio Spurs,0022400146,2024-11-02,SAS vs. MIN,None,120,74,...,0.889,1.0,20.0,21,20,4.0,3,6,7,5.0
2,22024,1610612766,CHA,Charlotte Hornets,0022400141,2024-11-02,CHA vs. BOS,L,239,103,...,0.667,7.0,27.0,34,24,10.0,1,13,25,-10.0
3,22024,1610612745,HOU,Houston Rockets,0022400144,2024-11-02,HOU vs. GSW,None,121,49,...,0.800,9.0,15.0,24,13,2.0,4,6,15,-28.0
4,22024,1610612761,TOR,Toronto Raptors,0022400143,2024-11-02,TOR vs. SAC,None,120,81,...,0.765,4.0,19.0,23,21,8.0,1,10,15,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123269,21983,1610612737,ATL,Atlanta Hawks,0028300005,1983-10-28,ATL @ NJN,L,240,108,...,0.684,15.0,18.0,33,14,10.0,5,17,30,NaN
123270,21983,1610612743,DEN,Denver Nuggets,0028300009,1983-10-28,DEN vs. UTH,W,240,139,...,0.744,19.0,37.0,56,40,4.0,8,11,26,NaN
123271,21983,1610612739,CLE,Cleveland Cavaliers,0028300008,1983-10-28,CLE vs. NYK,L,240,106,...,0.650,18.0,17.0,35,22,9.0,8,18,25,NaN
123272,21983,1610612765,DET,Detroit Pistons,0028300006,1983-10-28,DET vs. BOS,W,240,127,...,0.843,17.0,27.0,44,26,8.0,7,12,33,NaN


In [15]:
### CHECK TABLES
conn = allintwo.connect_to_rds('thebigone', 'ajwin', 'CharlesBark!23', 'nba-rds-instance.c9wwc0ukkiu5.us-east-1.rds.amazonaws.com')

cursor = conn.cursor()

cursor.execute("""
        SELECT tablename
        FROM pg_tables
        WHERE schemaname = 'public';
    """)


cursor.fetchall()

Connected to RDS PostgreSQL database


[('masterplayers',), ('mastergames',)]

In [13]:
### DROP TABLES
conn = allintwo.connect_to_rds('thebigone', 'ajwin', 'CharlesBark!23', 'nba-rds-instance.c9wwc0ukkiu5.us-east-1.rds.amazonaws.com')
cursor = conn.cursor()
cursor.execute("DROP TABLE IF EXISTS boxscoreadvancedv3_0;")
conn.commit()
conn.close()

Connected to RDS PostgreSQL database


In [16]:
query = "SELECT * FROM boxscoreadvancedv3_0;"  # Example query: fetch 10 rows from the 'games' table

result_df = allintwo.query_database_to_dataframe(conn, query)

result_df

c:\Users\ajwin\Projects\NBA\thebigone\allintwo.py:103: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dataframe = pd.read_sql(query, conn)


,gameid,teamid,teamcity,teamname,teamtricode,teamslug,personid,firstname,familyname,namei,...,turnoverratio,effectivefieldgoalpercentage,trueshootingpercentage,usagepercentage,estimatedusagepercentage,estimatedpace,pace,paceper40,possessions,pie
